In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [ ]:
print(train.info(verbose=True))

In [ ]:
max_columns = [x for x in train.columns if (x.endswith('max') and not(x.endswith('noninvasive_max')))]
noninvasive_min = [x for x in train.columns if (x.endswith('noninvasive_min'))]
apache = [x for x in train.columns if (x.endswith('apache'))]
train = train.drop(columns=max_columns)
train = train.drop(columns=noninvasive_min) 
train = train.drop(columns=apache)
print(train.info())

In [ ]:
train['hospital_id'] = train['hospital_id'].astype('object',copy=False)
test['hospital_id'] = test['hospital_id'].astype('object',copy=False)
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)
print(train_categorical_cols)

In [ ]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [ ]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]
inputcols.append('hospital_id')
print(inputcols)

In [ ]:
X = train[inputcols]
y = train[outputcols]

In [ ]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 )

In [ ]:
predictions = []

for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.04,depth = 10, loss_function='Logloss', iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])

In [ ]:
prediction = np.mean(predictions, axis=0)

In [ ]:
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("2501.csv",index=False)